# Generating Simulated Nanoparticle Images

In this notebook, simulated nanoparticles will be generated. These will then be rotated using rotation matrices and projected onto a 2D plane. This projection will be blurred and Poisson noise will be added. The projections will be saved as images directly into Google Drive, separated into training, validation, and testing datasets.

# Importing the relevant libraries and configuring the figure.

In [42]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import datetime

img_width, img_height = 128, 128
plt.style.use("dark_background")
fig = plt.figure(figsize=(3, 3))

<Figure size 216x216 with 0 Axes>

The images will be saved directly in Google Drive.

In [43]:
from google.colab import drive
drive_path = "/content/drive"
drive.mount(drive_path)

cwd = os.path.join(drive_path, "MyDrive", "Nanoparticles")
file_type = "png"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Creating the class.

The shape3D class generates a lattice to simulate different nanoparticle shapes. These bodies can be rotated in 3 dimensions and then projected onto a 2D plane. This plane can then have noise added to simulate Poisson noise from measuring instruments. This is then saved as an image. These functions will be shared for all lattice shapes.

In [44]:
class shape3D:
  """
  This class creates a 3D shape from a specified width and 3 angles of rotation, rx, ry, rz. The
  shape can then be projected onto 2 axes after being rotated. These projections are saved as
  images, and they can also have a Gaussian blur applied to them and those are saved in a
  subfolder.
  """
  def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
    """
    Initialises the class.

    Inputs:
      width: Integer, width of the shape being created
      rx, ry, rz: Float, rotations in degrees about the x, y, z axes respectively
      a: Float, lattice spacing
      defect: Float between 0 and 1, percentage of defects in the shape, 0 is no defects
      pos_error: Float, error in the position of each point
    """
    self.width = width
    self.rx = rx
    self.ry = ry
    self.rz = rz
    self.defect = defect
    self.pos_error = pos_error
    self.a = a
    self.alpha = 0.5

    data = "W%s RX%s RY%s RZ%s D%s" % (self.width, self.rx, self.ry, self.rz, int(100*self.defect))
    self.name = "%s %s" % (self.name, data)

    self.generate() # Generating the shape
    self.rotate() # Rotating the shape
    self.projection2D() # Projecting the shape onto a 2D surface

  def generate(self):
    """
    Subclasses override this function to generate self.coords.
    """
    pass

  def rotate(self):
    """
    Calculates the rotation matrix to rotate the shape.
    """
    # Converting degrees to radians
    rx = np.deg2rad(self.rx)
    ry = np.deg2rad(self.ry)
    rz = np.deg2rad(self.rz)

    # Rotation matrices
    Rx = np.array([[           1,           0,           0],
                    [           0,  np.cos(rx), -np.sin(rx)],
                    [           0,  np.sin(rx),  np.cos(rx)]])

    Ry = np.array([[  np.cos(ry),           0,  np.sin(ry)],
                    [           0,           1,           0],
                    [ -np.sin(ry),           0,  np.cos(ry)]])

    Rz = np.array([[  np.cos(rz), -np.sin(rz),           0],
                    [  np.sin(rz),  np.cos(rz),           0],
                    [           0,           0,           1]])

    # Applying the rotation
    self.R = np.matmul(np.matmul(Rz, Ry), Rx)
    self.coords = np.matmul(self.R, self.coords)

    self.rotation = self.R[:,0:2].T.reshape((1, 6))

  def projection2D(self):
    """
    Projects 3D shape onto 2D plane.
    """
    # Removing z axis
    self.coords2D = self.coords[0:, :]
    coords2D_shape = np.shape(self.coords2D)

    # Adding defects to the shape
    if self.defect != 0:
        iters = int(self.defect * coords2D_shape[0])

        for _ in range(iters):
            self.coords2D = np.delete(self.coords2D, np.random.randint(coords2D_shape[0]-1), 0)

    # Translating the shape around randomly
    # self.coords2D[0, :] += np.random.uniform(-self.a, self.a) * 2.4
    # self.coords2D[1, :] += np.random.uniform(-self.a, self.a) * 2.4

    # Creating the plot
    self.ax = fig.add_subplot(1, 1, 1)
    self.ax.set_axis_off()

    lim = 10
    self.lims = [-lim, lim]
    self.ax.set_xlim(self.lims)
    self.ax.set_ylim(self.lims)
    self.ax.set_aspect('equal', adjustable='box')
    self.ax.scatter(self.coords2D[0, :], self.coords2D[1, :], s=0.8, c="white", alpha=self.alpha)

  def save_projection(self, save_path=os.path.join(os.getcwd(), "Simulated Data"), file_name = "default", file_type=".png", blur=4, iter=0):
    """
    Saves the 2D projection.
    
    Inputs:
      save_path: String, folder in which to save the projections
      file_name: String, desired name for the projection's file name
      file_type: String, what file type to save the file as
      blur: Integer, strength of Gaussian blur
      iter: Integer, number of cubes with the same properties + 1
    """
    if not os.path.exists(save_path):
      os.makedirs(save_path)

    if file_name == "default":
      file_name = self.name + (" %s.%s" % (iter, file_type))

    # Creating the image from the canvas
    fig.canvas.draw()
    img = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    img = img.reshape(fig.canvas.get_width_height()[::-1] + (3, ))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    # Cropping the outer borders
    factor = 0.15
    low_y = int(img.shape[0] * factor)
    high_y = int(img.shape[0] - low_y)
    low_x = int(img.shape[1] * factor)
    high_x = int(img.shape[1] - low_x)

    cropped = img[low_x:high_x, low_y:high_y]

    # Blurring the projection
    kernel = np.ones((blur,blur),np.float32)/blur**2
    dst = cv2.filter2D(cropped,-1,kernel)

    dst = cv2.resize(dst, (img_width, img_height))

    # Where to save
    blur_name = file_name.split(".")[0] + (" B%s.%s" % (blur, file_type))
    rotation6d = self.rotation.tolist()[0]
    blur_path = os.path.join(save_path, "%s" % (rotation6d))
    blur_file_name = os.path.join(blur_path, blur_name)

    if not os.path.exists(blur_path):
      os.makedirs(blur_path)

    # Generating noise over the blurred image
    plt.cla()
    self.ax.imshow(dst, extent=[0, img_width, 0, img_height])
    noise_param = 6000
    noise_x = np.random.uniform(self.lims[0], self.lims[1], (noise_param))
    noise_y = np.random.uniform(self.lims[0], self.lims[1], (noise_param))
    self.ax.scatter(noise_x, noise_y, s=1, c="white", alpha=0.04)

    # Converting the canvas to image then saving
    fig.canvas.draw()
    dst = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    dst = dst.reshape(fig.canvas.get_width_height()[::-1] + (3, ))
    dst = cv2.cvtColor(dst, cv2.COLOR_RGB2GRAY)
    cv2.imwrite(blur_file_name, cv2.resize(dst[low_x:high_x, low_y:high_y], (img_width, img_height)))

    fig.clf()

Creating the subclasses for specific shapes which inherits from the shape3D class.

In [45]:
class cube3D(shape3D):
    """
    This class inherits from the shape3D class and is specifically for a cube.
    """
    def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
        """
        Create a cube and include that in the object name.
        """
        self.name = "Cube"
        super().__init__(width, rx, ry, rz, a, defect, pos_error)

    def generate(self):
        """
        Generates all xyz coordinates.
        """
        x0 = self.a * np.arange(-(self.width-1)/2, self.width/2, 1)
        self.coords = np.reshape(np.meshgrid(x0, x0, x0), (3, -1))
        self.coords += np.random.normal(0, self.pos_error, np.shape(self.coords)) * self.a

class tetra3D(shape3D):
    """
    This class inherits from the shape3D class and is specifically for a tetrahedron.
    """
    def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
        """
        Create a tetrahedron and include that in the object name.
        """
        self.name = "Tetrahedron"
        super().__init__(width, rx, ry, rz, a, defect, pos_error)

    def generate(self):
        """
        Generates all xyz coordinates.
        """
        shape_folder = os.path.join(cwd, "Vertices", "Tetrahedron")
        shape_txt = "Tetra%s_Verts.txt" % self.width
        self.coords = np.loadtxt(os.path.join(shape_folder, shape_txt))
        self.coords *= self.a
        self.coords += np.random.normal(0, self.pos_error, np.shape(self.coords)) * self.a

class octa3D(shape3D):
    """
    This class inherits from the shape3D class and is specifically for a octahedron.
    """
    def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
        """
        Create a octahedron and include that in the object name.
        """
        self.name = "Octahedron"
        super().__init__(width, rx, ry, rz, a, defect, pos_error)

    def generate(self):
        """
        Generates all xyz coordinates.
        """
        shape_folder = os.path.join(cwd, "Vertices", "Octahedron")
        shape_txt = "Octa%s_Verts.txt" % self.width
        self.coords = np.loadtxt(os.path.join(shape_folder, shape_txt))
        self.coords *= self.a
        self.coords += np.random.normal(0, self.pos_error, np.shape(self.coords)) * self.a

# Generating the images.

First, the folder in which the images will be saved must be specified.

In [46]:
folder_name = "Tetras Unmoving W12 RXRY"
dataset_path = os.path.join(cwd, folder_name)

The following function generates images for all combinations of widths, rotations and defects for a specified number of iterations. 

In [47]:
def generate_imgs(shape_class, w, rx, ry, rz, a, d, iters, blur, save_path, file_type):
  total_images = len(w) * len(rx) * len(ry) * len(rz) * len(d) * iters
  counter = 1
  for angle_z in rz:
    for angle_y in ry:
      for angle_x in rx:
        for width in w:
          for defect in d:
            for i in range(iters):
              print("\rGenerating image %s of %s." % (counter, total_images), end='', flush=True)
              shape = shape_class(width, angle_x, angle_y, angle_z, a=a, defect=defect, pos_error=np.random.uniform(0.03, 0.04))
              shape.save_projection(save_path=save_path, file_type=file_type, blur=blur, iter=i)

              counter += 1

Configuring the properties of the lattices to be generated.

In [48]:
# Properties of the shape and its projection
shape_class = tetra3D
width = [i for i in range(12, 16, 4)]
rx = [i/2 for i in range(0, 50, 5)] # Angle in degrees
ry = [i/2 for i in range(0, 50, 5)]
rz = [i for i in range(0, 50, 50)]

a = 1 # Pt nanoparticle lattice spacing: 0.24nm, atom size is ~0.1nm | 0.75 for cube
d = [i/100 for i in range(0, 25, 5)] # Percentage defect of the shape
blur = 15 # Strength of Gaussian blur

iters = 4

Generating the training, validation, and testing images.

In [49]:
begin_time = datetime.datetime.now()

# Generating the training images
save_path_train = os.path.join(dataset_path, "Train")
iters_train = 4 * iters

print("\nNow generating training dataset images...")
generate_imgs(
    shape_class=shape_class,
    w=width,
    rx=rx,
    ry=ry,
    rz=rz,
    a=a,
    d=d,
    iters=iters_train,
    blur=blur,
    save_path=save_path_train,
    file_type=file_type
)

# Generating the validation images
save_path_valid = os.path.join(dataset_path, "Valid")
iters_valid = iters

print("\nNow generating validation dataset images...")
generate_imgs(
  shape_class=shape_class,
  w=width,
  rx=rx,
  ry=ry,
  rz=rz,
  a=a,
  d=d,
  iters=iters_valid,
  blur=blur,
  save_path=save_path_valid,
  file_type=file_type
)

# Generating the testing images
save_path_test = os.path.join(dataset_path, "Test")
iters_test = iters

print("\nNow generating testing dataset images...")
generate_imgs(
  shape_class=shape_class,
  w=width,
  rx=rx,
  ry=ry,
  rz=rz,
  a=a,
  d=d,
  iters=iters_test,
  blur=blur,
  save_path=save_path_test,
  file_type=file_type
)

plt.close()
print("\nDone! The time it took is %s seconds." % round((datetime.datetime.now() - begin_time).total_seconds(), 2))


Now generating training dataset images...
Generating image 8000 of 8000.
Now generating validation dataset images...
Generating image 2000 of 2000.
Now generating testing dataset images...
Generating image 2000 of 2000.
Done! The time it took is 1189.93 seconds.
